In [2]:
from shutil import copyfile
import os
def split_dataset(source, training, testing, validation, split_size):

  file_names = os.listdir(source) 
  file_number = len(file_names) 

  for index, file in enumerate(file_names): 
      if index < split_size * file_number: 
        if index < 0.8 * file_number:
          copyfile(os.path.join(source, file), os.path.join(training, file))
        else:
          copyfile(os.path.join(source, file), os.path.join(validation, file))
      else: 
          copyfile(os.path.join(source, file), os.path.join(testing, file))

In [5]:
new_train_dir = r'D:\BRAIT\train'
new_test_dir = r'D:\BRAIT\test'
new_val_dir = r'D:\BRAIT\validation'

os.mkdir(new_train_dir)
os.mkdir(new_test_dir)
os.mkdir(new_val_dir)

In [6]:
source_dataset = r'D:\BRAIT\Braille Dataset'
dataset_list = os.listdir(source_dataset)

for data in dataset_list:
  print(data, " : ", str(len(os.listdir(os.path.join(source_dataset, data)))))
  os.mkdir(os.path.join(new_train_dir, data))
  os.mkdir(os.path.join(new_test_dir, data))
  os.mkdir(os.path.join(new_val_dir, data))

a  :  198
b  :  227
c  :  194
colon  :  59
d  :  240
e  :  125
f  :  124
g  :  125
h  :  125
i  :  125
j  :  125
k  :  125
koma  :  51
l  :  126
m  :  126
n  :  125
o  :  125
p  :  125
petik_satu  :  57
q  :  125
r  :  125
s  :  126
semicolon  :  58
space  :  15
t  :  125
tanda_hubung  :  54
tanda_seru  :  50
tanda_tanya  :  55
titik  :  54
u  :  126
v  :  125
w  :  125
x  :  125
y  :  125
z  :  125


In [7]:
split_size = .9

for dt in dataset_list:
  split_dataset(os.path.join(source_dataset, dt), os.path.join(new_train_dir, dt), os.path.join(new_test_dir, dt), os.path.join(new_val_dir, dt), split_size)

In [8]:
import os

In [102]:
path_train = r'D:\BRAIT\train'
path_test = r'D:\BRAIT\test'
path_val = r'D:\BRAIT\validation'

jmh_braille_train = os.listdir(path_train)
jmh_braille_test = os.listdir(path_test)
jmh_braille_val = os.listdir(path_val)

total_images = 0

for situs in jmh_situs_train:
  print(situs, " \t\t\t: ", len(os.listdir(os.path.join(path_train, situs)))+len(os.listdir(os.path.join(path_test, situs)))+len(os.listdir(os.path.join(path_val, situs))), " images")
  total_images += int(len(os.listdir(os.path.join(path_train, situs))))

print("Training Images : " + str(total_images))

a  			:  198  images
b  			:  227  images
c  			:  194  images
colon  			:  59  images
d  			:  240  images
e  			:  125  images
f  			:  124  images
g  			:  125  images
h  			:  125  images
i  			:  125  images
j  			:  125  images
k  			:  125  images
koma  			:  51  images
l  			:  126  images
m  			:  126  images
n  			:  125  images
o  			:  125  images
p  			:  125  images
petik_satu  			:  57  images
q  			:  125  images
r  			:  125  images
s  			:  126  images
semicolon  			:  58  images
space  			:  15  images
t  			:  125  images
tanda_hubung  			:  54  images
tanda_seru  			:  50  images
tanda_tanya  			:  55  images
titik  			:  54  images
u  			:  126  images
v  			:  125  images
w  			:  125  images
x  			:  125  images
y  			:  125  images
z  			:  125  images
Training Images : 3259


In [103]:
from keras_preprocessing.image import ImageDataGenerator

TARGET_SIZE = (150,100)
BATCH_SIZE = 32

TRAINING_DIR = r'D:\BRAIT\train'
training_datagen = ImageDataGenerator(
      rescale = 1./255,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

TESTING_DIR = r'D:\BRAIT\test'
testing_datagen = ImageDataGenerator(rescale=1./255)

VALIDATION_DIR = r'D:\BRAIT\validation'
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=TARGET_SIZE,
	class_mode='categorical',
  batch_size=BATCH_SIZE,
  shuffle = True
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=TARGET_SIZE,
	class_mode='categorical',
  batch_size=BATCH_SIZE,
  shuffle=True
)

test_generator = testing_datagen.flow_from_directory(
    TESTING_DIR,
    target_size=TARGET_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 3259 images belonging to 35 classes.
Found 417 images belonging to 35 classes.
Found 389 images belonging to 35 classes.


In [104]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    # The third convolution
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(37, activation='softmax')
])

In [105]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_708 (Conv2D)         (None, 148, 98, 32)       896       
                                                                 
 max_pooling2d_78 (MaxPooli  (None, 74, 49, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_709 (Conv2D)         (None, 72, 47, 32)        9248      
                                                                 
 max_pooling2d_79 (MaxPooli  (None, 36, 23, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_21 (Dropout)        (None, 36, 23, 32)        0         
                                                                 
 conv2d_710 (Conv2D)         (None, 34, 21, 64)      

In [110]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [111]:
history = model.fit(train_generator, 
                    epochs=15, 
                    steps_per_epoch=len(train_generator), 
                    validation_data = validation_generator, 
                    verbose = 1, 
                    validation_steps=len(validation_generator))

Epoch 1/15


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main

  File "C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code

  File "C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 1077, in launch_instance

  File "C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\asyncio\base_events.py", line 601, in run_forever

  File "C